In [7]:
from api.models import *
from api.distance import haversine
from django.db.models import F
from api.reverse_geocode import get_locality


In [2]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [3]:
location = {
    "coord_longitude" : -66,
    "coord_latitude" : 45.3888,
    "range_meters" : 1000,
}

In [9]:
location.update({
    "locality" : get_locality(location['coord_longitude'], location['coord_latitude']) 
})
location

{'coord_longitude': -66,
 'coord_latitude': 45.3888,
 'range_meters': 1000,
 'locality': {'name': 'Rothesay',
  'google_place_id': 'ChIJ_2vsoe-sp0wRwi0gFylfEZs'}}

In [4]:
posts = Post.objects.annotate(
                    distance = haversine(F('coord_longitude'), F('coord_latitude'), location['coord_longitude'], location['coord_latitude'])
                ).filter(distance__lte=location['range_meters']).order_by('-created_on')

In [5]:
for i in posts:
    print(i.__dict__)

{'_state': <django.db.models.base.ModelState object at 0x000001E0971363E0>, 'id': 17, 'title': ':)', 'content': 'new_post', 'ref_url': '', 'image': '', 'author_id': 2, 'created_on': datetime.datetime(2022, 1, 20, 22, 51, 0, 115151, tzinfo=datetime.timezone.utc), 'coord_longitude': -65.9943, 'coord_latitude': 45.3888, 'range_meters': 10000, 'locality_id': 'ChIJ_2vsoe-sp0wRwi0gFylfEZs', 'extend_to_locality': False, 'distance': 445.1205835871303}
{'_state': <django.db.models.base.ModelState object at 0x000001E0971364A0>, 'id': 16, 'title': ':)', 'content': 'new_post', 'ref_url': '', 'image': '', 'author_id': 2, 'created_on': datetime.datetime(2022, 1, 20, 22, 50, 23, 607168, tzinfo=datetime.timezone.utc), 'coord_longitude': -65.9943, 'coord_latitude': 45.3888, 'range_meters': 10000, 'locality_id': 'ChIJ_2vsoe-sp0wRwi0gFylfEZs', 'extend_to_locality': False, 'distance': 445.1205835871303}
{'_state': <django.db.models.base.ModelState object at 0x000001E097136560>, 'id': 15, 'title': ':)', 'c

# Range Mode Query

In [6]:
posts = Post.objects.annotate(
            distance = haversine(F('coord_longitude'), F('coord_latitude'), location['coord_longitude'], location['coord_latitude'])
        ).filter(
            distance__lte=location['range_meters']
        ).filter(
            distance__lte=F('range_meters')   
        ).order_by('-created_on')

# Locality Mode Query

In [34]:
posts_extend = Post.objects.filter(
    extend_to_locality = True, locality__google_place_id = location['locality']['google_place_id']
)
posts_no_extend = posts = Post.objects.annotate(
    distance = haversine(F('coord_longitude'), F('coord_latitude'), location['coord_longitude'], location['coord_latitude'])
).filter(
    extend_to_locality = False, locality__google_place_id = location['locality']['google_place_id'], distance__lte = F('range_meters')
)
combined = (posts_extend | posts_no_extend).distinct()

combined


<QuerySet [<Post: :) - testUser1 - 2022-01-20 21:04:00.970947+00:00>, <Post: :) - testUser1 - 2022-01-20 22:50:23.607168+00:00>, <Post: :) - testUser1 - 2022-01-20 22:51:00.115151+00:00>]>

False